In [3]:
import gymnasium as gym
from stable_baselines3 import PPO
from config import config
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

%load_ext tensorboard
import sys
from tqdm.notebook import trange
# from utils import record_videos, show_videos

env_id = "intersection-v0"
# num_cpu = 4
env = gym.make(env_id, render_mode="rgb_array", config=config)
# env = make_vec_env(env_id, n_envs=num_cpu)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir "intersection_ppo/PPO_1"

In [6]:
ppo_params = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 64,
    'tensorboard_log':'intersection_ppo'
}

model = PPO("MlpPolicy", env, verbose=1,**ppo_params)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
# We create a separate environment for evaluation
eval_env = gym.make(env_id, render_mode="rgb_array", config=config)

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment intersection-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(
c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 4.976836729049682 +/- 4.76


In [8]:
model.learn(total_timesteps=1,progress_bar=True)
model.save("test_PPO_intersection_matthias")

Logging to intersection_ppo/PPO_1


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.8      |
|    ep_rew_mean     | 0.235    |
| time/              |          |
|    fps             | 4        |
|    iterations      | 1        |
|    time_elapsed    | 492      |
|    total_timesteps | 2048     |
---------------------------------


In [10]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 1.3 +/- 2.41


In [12]:
%matplotlib inline
import matplotlib.pyplot as plt

while True:
  done = truncated = False
  obs, info = env.reset()
  while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    env.render()



AttributeError: 'NoneType' object has no attribute 'get_image'

In [ ]:
env = gym.make('highway-fast-v0', render_mode='rgb_array')
env = record_videos(env)
for episode in trange(3, desc='Test episodes'):
    (obs, info), done = env.reset(), False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(int(action))
env.close()
show_videos()